### **0430 Qwen部分修改**

In [31]:
import pandas as pd

In [32]:
train = pd.read_csv('/root/LLM_news_emo_analyze/DATA/deepseek_sft_0419data/ds_train_data.csv',encoding = 'utf_8_sig',lineterminator='\n')
test = pd.read_csv('/root/LLM_news_emo_analyze/DATA/deepseek_sft_0419data/ds_valid_data.csv',encoding = 'utf_8_sig',lineterminator='\n')

In [33]:
prompt = """**角色定义**
你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的新闻文本情绪进行准确的评分：

**任务流程**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词，对情绪进行深入理解与分析
3. 输出情绪分析跟对应的五档制评分结果（-1.0/-0.5/0.0/0.5/1.0，分别代表非常消极、比较消极、中性、比较积极、非常积极）

**评分逻辑**
1. 语义匹配：忽略偏离的情绪词词汇，保留有效情绪词
2. 评分调整：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
情绪分析:①语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破；②关键情绪词调整：“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1.0分（非常积极）的评分结果。
情绪得分:1.0

- 示例2：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
情绪分析:①语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号；②关键情绪词调整：未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。由于缺乏词典强匹配项限制进一步上调空间，整体乐观基调符合“比较积极”，情绪得分0.5。
情绪得分:0.5

- 示例3：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
情绪分析:①语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响；②关键情绪词调整：“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0.0分。
情绪得分:0.0

- 示例4：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
情绪分析:①语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪；②关键情绪词调整：“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位“比较消极”，即情绪得分-0.5分。
情绪得分:-0.5

- 示例5：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
情绪分析:①语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词调整：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，因此是非常消极，情绪得分-1.0分。
情绪得分:-1.0


**其他说明**
- 情绪词典的分值仅作语义方向参考
- 情绪得分必须是五档制选择，不得出现-1.0/-0.5/0.0/0.5/1.0之外的分数
- 输出格式：{"情绪分析":"...","情绪得分":"..."}

现在，请你开始分析并按照要求输出结果：
新闻文本：{{新闻文本}}
情绪词典：{{情绪词}}"""

In [34]:
train['qwen_input'] = train.apply(
    lambda row: prompt.replace('{{新闻文本}}', row['cut_news'] + ('。' if not row['cut_news'].endswith('。') else ''))
              .replace('{{情绪词}}', row['sentiment_dict_v2']),
    axis=1
)

test['qwen_input'] = test.apply(
    lambda row: prompt.replace('{{新闻文本}}', row['cut_news'] + ('。' if not row['cut_news'].endswith('。') else ''))
              .replace('{{情绪词}}', row['sentiment_dict_v2']),
    axis=1
)


In [ ]:
# def extract_number(text):
#     pattern = r':(-?\d+(?:\.\d+)?)'
#     match = re.search(pattern, text)
#     if match:
#         return float(match.group(1))
#     return None

# # 应用函数到 content 列并创建新列 score
# data['score'] = data['content'].apply(extract_number)

In [35]:
import re

def extract_last_sentiment_score(text):
    # 匹配的情绪得分值（注意：用 r'\b' 保证是独立的数字）
    pattern = r'(?<!\d)(-1\.0|-0\.5|0\.5|1\.0|-1|0\.0|1|0)(?!\d)'
    matches = re.findall(pattern, text)
    if matches:
        return matches[-1]  # 返回最后一个匹配值
    return None  # 没有匹配则返回 None


In [36]:
train['final_score'] = train['Complex_CoT'].apply(extract_last_sentiment_score)

In [37]:
import pandas as pd

# 步骤1：统一格式（去除空格，转换为字符串再转为 float）
def normalize_score(val):
    try:
        return float(str(val).strip())
    except:
        return None  # 或者 np.nan，如果你想标记无效值

train['final_score_clean'] = train['final_score'].apply(normalize_score)
train['score_clean'] = train['score'].apply(normalize_score)

# 步骤2：计算不一致行（用 np.isclose 更稳健，考虑浮点误差）
import numpy as np

train['is_equal'] = np.isclose(train['final_score_clean'], train['score_clean'], atol=1e-6)

# 步骤3：计算不一致率
total = len(train)
inconsistent = (~train['is_equal']).sum()
inconsistency_rate = inconsistent / total

print(f'不一致率为：{inconsistency_rate:.2%}（{inconsistent}/{total}）')

# 步骤4：输出不一致的行
diff_rows = train[~train['is_equal']]
print("不一致的样例如下：")
print(diff_rows[['final_score', 'score']].head(30))  # 可根据需要显示更多行


不一致率为：4.78%（529/11065）
不一致的样例如下：
    final_score  score
37          0.5    0.0
171         0.5    0.0
177           1    0.5
190           1    0.5
209           1    0.5
218           1    0.5
219           1    0.5
299         0.5    0.0
317           0   -0.5
319         0.5    0.0
334         0.5    0.0
357           1    0.5
386         0.5    0.0
391         0.5    0.0
423           1    0.5
424         0.5    0.0
432           1    0.5
457         0.5    0.0
460         0.5    0.0
522           1    0.5
533         0.5    0.0
569           1    0.5
599         0.5    0.0
633         0.5    0.0
637         0.5    0.0
647           1    0.5
693           1    0.5
711           1    0.5
744         0.5    0.0
757           1    0.5


In [38]:
train = train.drop(columns=['final_score', 'score_clean', 'is_equal'])

In [43]:
train

,Title,NewsSource,stratum,topic,similarity,cut_news,sentiment_dict,sentiment_dict_v2,Response,Complex_CoT,score,Question,qwen_input,final_score_clean,qwen_output
0,下半场还是熊来了 牛市底盘裂缝,经济观察报,2008-03-17 08:23:00_经济观察报,“金融”,0.480989,易方达的一位基金经理表示，机构投资者在较早时期便开始减仓，目前市场更多表现为中小投资者的恐慌...,"撤资/抛售(divestment):-0.2,清仓/抛售(closeouts):-0.2","divestment:-0.2, closeouts:-0.2","{""情绪得分"": ""-1""}",好，我需要分析这篇新闻的情绪。新闻中提到易方达的基金经理说机构投资者早前就开始减仓，现在市场...,-1.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,-1.0,"{""情绪分析"":""好，我需要分析这篇新闻的情绪。新闻中提到易方达的基金经理说机构投资者早前就..."
1,风投的另类生意,经济日报,2014-05-02 00:00:00_经济日报,“金融”,0.491875,“风投采取PIPE投资能增加PE资本流动性，提高PE资本的变现能力,"资本/资金(capital):0.1,投资/投资额(investment):0.4","capital:0.1, investment:0.4","{""情绪得分"": ""0.5""}",好的，我来分析一下这个新闻文本的情绪。首先，新闻内容是“风投采取PIPE投资能增加PE资本流...,0.5,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,0.5,"{""情绪分析"":""好的，我来分析一下这个新闻文本的情绪。首先，新闻内容是“风投采取PIPE投..."
2,投资者结构分化中的贵州茅台：千元股价贵不贵？,第一财经日报,2019-06-24 00:00:00_第一财经日报,“金融”,0.524268,”深圳一位公募基金经理也表示,"基金/资金(fund):0.1,基础/底部(base):0.0","fund:0.1, base:0.0","{""情绪得分"": ""0""}",好的，我现在需要分析给定的新闻文本的情绪得分。首先，新闻文本是：“深圳一位公募基金经理也表示...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,0.0,"{""情绪分析"":""好的，我现在需要分析给定的新闻文本的情绪得分。首先，新闻文本是：“深圳一位..."
3,32次降价失败后 购销“两票制”能让药价瘦身吗,第一财经日报,2017-01-10 00:00:00_第一财经日报,“生产投资”,0.474199,这结果值得我们反思，根源是否就在于招采的制度和机制,"选矿/精选(beneficiation):0.4,管理不善/管理失误(mismanageme...","beneficiation:0.4, mismanagement:-0.5","{""情绪得分"": ""-0.5""}",好的，我需要分析这条新闻的情绪得分。首先看一下新闻内容：“这结果值得我们反思，根源是否就在于...,-0.5,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,-0.5,"{""情绪分析"":""好的，我需要分析这条新闻的情绪得分。首先看一下新闻内容：“这结果值得我们反..."
4,用实验探索贫穷的本质,经济观察报,2019-10-19 02:09:00_经济观察报,“宏观经济”,0.505570,这种人民币形态的热钱规模有多大,"美分/分币(cent):0.0,货币/通货(currency):0.0","cent:0.0, currency:0.0","{""情绪得分"": ""0""}",好的，我来分析一下这个新闻文本的情绪。首先，新闻内容是“这种人民币形态的热钱规模有多大。”这...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,0.0,"{""情绪分析"":""好的，我来分析一下这个新闻文本的情绪。首先，新闻内容是“这种人民币形态的热..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11060,退市新规精准发力：有效出清绩差风险公司 ST股数量总体稳定,21世纪经济报道,2024-06-06 00:00:00_21世纪经济报道,“金融”,0.594900,同时，坚持对包括退市公司在内的财务造假公司及相关责任人违法违规行为“一追到底”、依法严惩，今...,"应受惩罚的/可惩处的(punishable):-0.4,公司的/企业的(corporate)...","punishable:-0.4, corporate:0.0","{""情绪得分"": ""0""}",好的，我现在需要分析这段中文新闻的情绪，并给出五档制评分。首先，先仔细阅读新闻内容。\n\n...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,0.0,"{""情绪分析"":""好的，我现在需要分析这段中文新闻的情绪，并给出五档制评分。首先，先仔细阅读..."
11061,提升电气化水平 驱动经济社会高质量发展,经济参考报,2020-06-15 00:00:00_经济参考报,“生产投资”,0.488712,在城乡居民用电领域，加大北方地区清洁取暖工作力度，提升清洁能源供热能力，发挥电供暖示范作用,"北方/北部(north):0.0,电的/电气的(electrical):0.0","north:0.0, electrical:0.0","{""情绪得分"": ""0""}",好的，我需要分析这段关于中国城乡居民用电领域的新闻文本的情绪，并根据提供的评分标准给出五档制...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,0.5,"{""情绪分析"":""好的，我需要分析这段关于中国城乡居民用电领域的新闻文本的情绪，并根据提供的..."
11062,“浑水”如何搅起一池浑水,第一财经日报,2011-11-25 00:00:00_第一财经日报,“金融”,0.544560,业内人士透露，集体诉讼原告费用一般由律师预支，采取风险代理模式，如果诉讼成功，律师事务所能够...,"诉讼当事人(litigant):0.0,诉讼/官司(lawsuit):-0.2","litigant:0.0, lawsuit:-0.2","{""情绪得分"": ""-0.5""}",好，我需要分析这篇新闻的情绪，然后根据给定的情绪词典和评分标准来打分。首先，仔细阅读新闻内容...,-0.5,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,0.0,"{""情绪分析"":""好，我需要分析这篇新闻的情绪，然后根据给定的情绪词典和评分标准来打分。首先..."
11063,资金融通助力下的“一带一路”硕果,中国经济周刊,2023-10-31 15:48:00_中国经济周刊,“消费”,0.597310,此外，中国已与20个共建国家签署双边本币互换协议，在17个共建国家建立人民币清算安排，人民币...,"协议/协定(agreement):0.4,贸易/交易(trade):0.0","agreement:0.4, trade:0.0","{""情绪得分"": ""0.5""}",好的，我现在需要分析用户提供的新闻文本的市场情绪，并给出五档制的评分。首先，我要仔细阅读新闻...,0.5,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,1.0,"{""情绪分析"":""好的，我现在需要分析用户提供的新闻文本的市场情绪，并给出五档制的评分。首先..."


In [41]:
# 创建新列 prompt_qwen，进行替换操作
train['qwen_output'] = train.apply(lambda row: '{"情绪分析":"'+ str(row['Complex_CoT']).replace('"', '\\"') + '","情绪得分":"' + str((row['final_score_clean'])) + '"}', axis=1)

In [ ]:
train.iloc[11064,-1]

In [46]:
allowed_scores = [-1, -0.5, 0, 0.5, 1,1.0,-0.50,-0.50,-1.0]

# 筛选 score 列值在允许列表中的行
train2 = train[train['final_score_clean'].isin(allowed_scores)]

In [ ]:
# train2.to_csv('/root/LLM_news_emo_analyze/DATA/qwen_sft_train_0405data/qwen_train_data.csv',index = False)

In [47]:
print(len(train))
print(len(train2))

11065
11065


### **0430 deepseek部分字段修改**

In [48]:
prompt_ds = """**角色定义**
擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分，请思考后回答。

**核心任务**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词，对情绪进行深入理解与思考
3. 思考后直接输出对应的五档制评分结果（-1.0/-0.5/0.0/0.5/1.0），这些值分别代表非常消极、比较消极、中性、比较积极、非常积极）

**评分逻辑**
1. 语义匹配：忽略偏离词汇，保留有效情绪词
2. 评分验证：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1.0"}

- 示例2：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}

- 示例3：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0。</think> {"情绪得分":"0.0"}

- 示例4：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
输出:<think>语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪。新闻文本中“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位-0.5。</think> {"情绪得分":"-0.5"}

- 示例5：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
输出:<think>语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词验证：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，符合-1的情绪评分。</think> {"情绪得分":"-1.0"}

**输出规范**
- 情绪词典的分值仅作语义方向参考
- 情绪得分必须是五档制选择，不得出现-1.0/-0.5/0.0/0.5/1.0之外的分值
- 输出格式：{"情绪得分":"..."}

现在请你开始进行分析，严格按照要求输出结果：
新闻文本：{cut_news}
情绪词典：{sentiment_dict_v2}"""

In [49]:
train['Question'] = train.apply(
    lambda row: prompt_ds.replace('{cut_news}', row['cut_news'] + ('。' if not row['cut_news'].endswith('。') else ''))
              .replace('{sentiment_dict_v2}', row['sentiment_dict_v2']),
    axis=1
)

test['Question'] = test.apply(
    lambda row: prompt_ds.replace('{cut_news}', row['cut_news'] + ('。' if not row['cut_news'].endswith('。') else ''))
              .replace('{sentiment_dict_v2}', row['sentiment_dict_v2']),
    axis=1
)

In [50]:
test.iloc[1,-1]

'**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评分，请思考后回答。\n\n**核心任务**\n1. 分析新闻整体内容情绪\n2. 匹配情绪词典关键词，对情绪进行深入理解与思考\n3. 思考后直接输出对应的五档制评分结果（-1.0/-0.5/0.0/0.5/1.0），这些值分别代表非常消极、比较消极、中性、比较积极、非常积极）\n\n**评分逻辑**\n1. 语义匹配：忽略偏离词汇，保留有效情绪词\n2. 评分验证：整体语义优先，情绪词辅助修正\n\n**示例说明**\n- 示例1：\n新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一\n情绪词典：profitability:0.6,profit:0.8\n输出:<think>语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破。“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1的评分逻辑。</think> {"情绪得分":"1.0"}\n\n- 示例2：\n新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力\n情绪词典：mispricing:-0.4,advantage:0.7\n输出:<think>语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号。此外，新闻文本未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。整体乐观基调符合中档评分0.5，缺乏词典强匹配项限制进一步上调空间。</think> {"情绪得分":"0.5"}\n\n- 示例3：\n新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动\n情绪词典：free:0.2,immateriality:-0.2\n输出:<think>语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响。此外，新闻文本中“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateri

In [51]:
import os
import uuid
import json
def create_json_dataset(train_origin_data):
    df = train_origin_data
    dataset = []
    for _, row in df.iterrows():
        item = {
            "id": str(uuid.uuid4()),
            "messages": [
                {
                    "role": "user",
                    "content": row['qwen_input']
                },
                {
                    "role": "assistant",
                    "content": row['qwen_output']
                }
            ]
        }
        dataset.append(item)
    print(len(dataset))
    return dataset


def save_to_json(dataset, output_path):
    # 获取文件所在的目录
    dir_path = os.path.dirname(output_path)
    # 检查目录是否存在，如果不存在则创建
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)


if __name__ == "__main__":
    output_json_path = '/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/qwen_train_data.json'  # 请替换为实际的输出 JSON 文件路径

    json_dataset = create_json_dataset(train)
    save_to_json(json_dataset, output_json_path)
    print(f"JSON 数据集已成功保存到 {output_json_path}")
    

11065
JSON 数据集已成功保存到 /root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/qwen_train_data.json


In [53]:
# 计数分布
print("各分数的数量分布：")
print(train['final_score_clean'].value_counts(dropna=False))

# 占比分布（百分比）
print("\n各分数的占比分布：")
print(train['final_score_clean'].value_counts(normalize=True, dropna=False).apply(lambda x: f"{x:.2%}"))


各分数的数量分布：
final_score_clean
 0.0    4499
 0.5    2754
-0.5    2001
 1.0    1160
-1.0     651
Name: count, dtype: int64

各分数的占比分布：
final_score_clean
 0.0    40.66%
 0.5    24.89%
-0.5    18.08%
 1.0    10.48%
-1.0     5.88%
Name: proportion, dtype: object


In [54]:
train.to_csv('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/both_train_data.csv',encoding = 'utf_8_sig',index = False)
test.to_csv('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/both_test_data.csv',encoding = 'utf_8_sig',index = False)

### **模型合并**

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
from transformers.generation.utils import GenerationConfig

 
def apply_lora(model_name_or_path, output_path, lora_path):
    print(f"Loading the base model from {model_name_or_path}")
    base_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False, trust_remote_code=True)
    base = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="cuda:0", torch_dtype=torch.bfloat16, trust_remote_code=True)
    # base.generation_config = GenerationConfig.from_pretrained(model_name_or_path)

    print(f"Loading the LoRA adapter from {lora_path}")
 
    lora_model = PeftModel.from_pretrained(
        base,
        lora_path,
        torch_dtype=torch.float16,
    )
 
    print("Applying the LoRA")
    model = lora_model.merge_and_unload()
 
    print(f"Saving the target model to {output_path}")
    model.save_pretrained(output_path)
    base_tokenizer.save_pretrained(output_path)


if __name__ == "__main__":
    lora_path = "/root/LLM_news_emo_analyze/Qwen_output/0430_lora_model/checkpoint-2800"
    model_path = "/root/.cache/LLMS/hub/Qwen/Qwen2___5-1___5B-Instruct"
    output = "/root/LLM_news_emo_analyze/Qwen_output/0430_merge_model"

    apply_lora(model_path,output,lora_path)

### **单个样本测试**

In [4]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from tqdm import tqdm

# 模型路径配置
#model_name = "/root/.cache/LLMS/hub/Qwen/sft_model/checkpoint-2000"
model_name = "/root/LLM_news_emo_analyze/Qwen_output/0430_merge_model"

# 初始化模型和分词器（优化后的加载方式）
try:
    # 加载分词器时显式指定padding方向
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        padding_side='left',  # 确保生成式任务的填充方向正确
        trust_remote_code=True  # 网页12、17提到需要信任远程代码
    )
    
    # 自动选择计算精度和设备分布
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",  # 网页1、4、12建议使用auto设备映射
        max_memory={i: '20GB' for i in range(torch.cuda.device_count())},  # 显存优化（网页5、8）
        trust_remote_code=True
    ).eval()
    
except Exception as e:
    print(f"初始化失败: {str(e)}")
    exit(1)

def generate_response(prompt, max_length=512, temperature=0.4):
    """优化后的生成函数（参考网页12、18）"""
    try:
        # 构建消息模板（网页13、14的ChatML格式）
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
        
        # 应用聊天模板（网页12、14最佳实践）
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True  # 添加<|im_start|>assistant\n前缀
        )
        
        model_inputs = tokenizer(
            [text],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048
        ).to(model.device)
        
        generation_config = {
            "max_new_tokens": 2048,
            "temperature": temperature,
            "top_p": 0.9,
            "repetition_penalty": 1.1,
            "pad_token_id": tokenizer.eos_token_id  # 显式设置pad_token（网页8、9）
        }
        
        # 执行生成
        generated_ids = model.generate(
            **model_inputs,
            **generation_config
        )
        
        response_ids = [output_ids[len(input_ids):] 
                       for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        
        return tokenizer.batch_decode(response_ids, skip_special_tokens=True)[0]
    
    except Exception as e:
        print(f"生成失败: {str(e)}")
        return None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
prompt_template = """**角色定义**
你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的新闻文本情绪进行准确的评分：

**任务流程**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词，对情绪进行深入理解与分析
3. 输出情绪分析跟对应的五档制评分结果（-1.0/-0.5/0.0/0.5/1.0，分别代表非常消极、比较消极、中性、比较积极、非常积极）

**评分逻辑**
1. 语义匹配：忽略偏离的情绪词词汇，保留有效情绪词
2. 评分调整：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
情绪分析:①语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破；②关键情绪词调整：“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1.0分（非常积极）的评分结果。
情绪得分:1.0

- 示例2：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
情绪分析:①语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号；②关键情绪词调整：未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。由于缺乏词典强匹配项限制进一步上调空间，整体乐观基调符合“比较积极”，情绪得分0.5。
情绪得分:0.5

- 示例3：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
情绪分析:①语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响；②关键情绪词调整：“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0.0分。
情绪得分:0.0

- 示例4：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
情绪分析:①语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪；②关键情绪词调整：“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位“比较消极”，即情绪得分-0.5分。
情绪得分:-0.5

- 示例5：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
情绪分析:①语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词调整：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，因此是非常消极，情绪得分-1.0分。
情绪得分:-1.0


**其他说明**
- 情绪词典的分值仅作语义方向参考
- 情绪得分必须是五档制选择，不得出现-1.0/-0.5/0.0/0.5/1.0之外的分数
- 输出格式：{"情绪分析":"...","情绪得分":"..."}

现在，请你开始分析并按照要求输出结果：
新闻文本：发展大飞机对国民经济增长具有带动效应，大飞机产业链蓄势待发
情绪词典：growth:0.7, productive:0.7"""

In [8]:
generate_response(prompt_template)

'{"情绪分析":"好的，我现在需要处理这个用户的查询，分析给定的新闻文本的情绪得分。首先，用户给了一个中文新闻文本，并且有一个情绪词典，里面有两个关键词：growth和productive，都是0.7分。\n\n首先，我得仔细阅读新闻内容：“发展大飞机对国民经济增长具有带动效应，大飞机产业链蓄势待发。” 这句话的关键点在于“带动效应”和“蓄势待发”。这里的“带动效应”明显是一个正面的描述，说明大飞机的发展会促进经济增长，而“蓄势待发”则暗示着未来会有更多的进展和机会，属于积极的信号。\n\n接下来要检查情绪词典里的关键词是否出现在新闻中。用户提供的情绪词典里有growth和productive，这两个词在新闻中确实出现了，原文中的“经济增长”对应了growth，“蓄势待发”可能没有直接出现productive这个词，但“蓄势待发”可以视为类似的意思，即准备充分，等待时机，这可能与productive相关，不过需要看是否有更直接的匹配。根据示例中的情况，比如示例2中的“便宜”虽然没直接匹配mispricing，但通过隐含意义进行了映射，所以这里可能类似，但需要确认是否严格按词典匹配。\n\n然后按照评分逻辑，首先是语义匹配，忽略不相关的词汇，只保留有效情绪词。这里的关键词是growth和productive，两个都是积极的，而且都对应到新闻中的词汇。“带动效应”中的“增长”对应growth，“蓄势待发”可能部分匹配productive，但productive更多指生产效率高，而这里的大飞机产业链蓄势待发更多是说准备好了，可能还没有实际产出，所以可能不算完全匹配。\n\n接下来是评分验证，整体语义优先。整个句子都在强调大飞机带来的积极影响，无论是经济增长还是产业链的蓄势待发，都是明显的正面消息。情绪词的存在加强了这一点，尤其是growth的0.7分，属于较高的积极分数。\n\n再看看提供的示例，例如示例1中，净利润大幅增长，直接匹配到了profit和profitability，得到了1分。而当前案例中的关键词虽然匹配到了，但可能因为不是完全相同的词，或者程度不如示例1那么强烈。例如，这里的“带动效应”可能比单纯的利润增长更积极，但示例2中的情况是间接映射，得到0.5分。但在这个例子中，直接提到了growth，而且整体语义非常积极，可能应该更高。\n\n不过需要注意的是

### **批量样本测试**

In [ ]:
import pandas as pd
df = pd.read_csv("/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/both_test_data.csv", encoding='utf_8_sig', lineterminator='\n')

In [ ]:
import os
import torch
import logging
import pandas as pd
from typing import List, Dict, Union, Optional
from modelscope import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 从环境变量或默认值获取模型路径
DEFAULT_MODEL_PATH = "/root/LLM_news_emo_analyze/Qwen_output/0430_merge_model_2100"
MODEL_PATH = os.environ.get("QWEN_MODEL_PATH", DEFAULT_MODEL_PATH)

class QwenInferenceEngine:
    """Qwen模型推理引擎，支持批量处理"""
    
    def __init__(self, model_path: str = MODEL_PATH):
        """初始化模型和分词器"""
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                model_path,
                padding_side='left',
                trust_remote_code=True
            )
            
            # 检测可用GPU并设置合适的精度
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            if self.device == "cpu":
                logger.warning("未检测到GPU，将使用CPU进行推理，性能可能受限")
                torch_dtype = torch.float32
            else:
                logger.info(f"检测到{torch.cuda.device_count()}个GPU设备")
                torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
            
            # 动态分配GPU内存
            max_memory = {}
            if torch.cuda.is_available():
                for i in range(torch.cuda.device_count()):
                    gpu_total_mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)  # GB
                    max_memory[i] = f"{int(gpu_total_mem * 0.85)}GB"  # 使用85%的GPU内存
            
            self.model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch_dtype,
                device_map="auto",
                max_memory=max_memory if max_memory else None,
                trust_remote_code=True
            ).eval()
            
            # 验证模型加载状态
            self._validate_model()
            
            logger.info(f"成功加载Qwen模型，设备:{self.device}，精度:{torch_dtype}")
            
        except Exception as e:
            logger.error(f"模型初始化失败: {str(e)}")
            raise RuntimeError(f"模型初始化失败: {str(e)}")
    
    def _validate_model(self):
        """验证模型是否正常加载和工作"""
        try:
            # 尝试进行一次简单推理
            test_input = "Hello"
            input_ids = self.tokenizer(test_input, return_tensors="pt").input_ids.to(self.model.device)
            self.model.generate(input_ids, max_new_tokens=5)
            logger.info("模型验证通过")
        except Exception as e:
            logger.error(f"模型验证失败: {str(e)}")
            raise RuntimeError(f"模型验证失败: {str(e)}")
    
    def generate_response(
        self, 
        prompt: str, 
        max_new_tokens: int = 2048, 
        temperature: float = 0.4,
        top_p: float = 0.9,
        repetition_penalty: float = 1.1
    ) -> Optional[str]:
        """单条输入的推理函数，保持向后兼容性"""
        results = self.generate_batch([prompt], max_new_tokens, temperature, top_p, repetition_penalty)
        return results[0] if results else None
    
    def generate_batch(
        self, 
        prompts: List[str], 
        max_new_tokens: int = 2048, 
        temperature: float = 0.4,
        top_p: float = 0.9,
        repetition_penalty: float = 1.1,
        batch_size: int = 15  # 控制单批次最大数量
    ) -> List[Optional[str]]:
        """批量处理多个输入的推理函数"""
        if not prompts:
            logger.warning("收到空输入列表")
            return []
        
        results = []
        
        # 对较大的输入列表分批处理
        for i in range(0, len(prompts), batch_size):
            batch_prompts = prompts[i:i+batch_size]
            batch_results = self._process_single_batch(
                batch_prompts, 
                max_new_tokens, 
                temperature,
                top_p,
                repetition_penalty
            )
            results.extend(batch_results)
            
        return results
    
    def _process_single_batch(
        self, 
        batch_prompts: List[str],
        max_new_tokens: int,
        temperature: float,
        top_p: float,
        repetition_penalty: float
    ) -> List[Optional[str]]:
        """处理单批次输入"""
        try:
            # 构建消息模板
            batch_messages = [
                [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ] for prompt in batch_prompts
            ]
            
            # 应用聊天模板
            batch_texts = [
                self.tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                ) for messages in batch_messages
            ]
            
            # 统一处理tokenization
            encodings = self.tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=2048,  # 控制输入最大长度
                return_attention_mask=True
            ).to(self.model.device)
            
            # 保存输入长度用于截取输出
            input_lengths = encodings.input_ids.shape[1]
            
            generation_config = {
                "max_new_tokens": max_new_tokens,
                "temperature": temperature,
                "top_p": top_p,
                "repetition_penalty": repetition_penalty,
                "pad_token_id": self.tokenizer.eos_token_id,
                "attention_mask": encodings.attention_mask,
                "do_sample": temperature > 0.0,  # 温度大于0使用采样
            }
            
            # 执行批量生成
            with torch.no_grad():  # 优化内存使用
                generated_ids = self.model.generate(
                    encodings.input_ids,
                    **generation_config
                )
            
            # 分别提取每个输入对应的生成结果
            batch_responses = []
            for i, output_ids in enumerate(generated_ids):
                # 提取生成的部分 (除去输入部分)
                response_ids = output_ids[len(encodings.input_ids[i]):]
                response_text = self.tokenizer.decode(response_ids, skip_special_tokens=True).strip()
                batch_responses.append(response_text)
            
            return batch_responses
            
        except Exception as e:
            error_msg = f"批处理生成失败: {str(e)}"
            logger.error(error_msg)
            # 返回对应数量的None，保持输出和输入一一对应
            return [None] * len(batch_prompts)

def process_csv_file(
    csv_path: str, 
    output_path: str = None,
    input_column: str = "qwen_input",
    output_column: str = "qwen_output",
    batch_size: int = 15,
    max_new_tokens: int = 2048,
    temperature: float = 0.4
):
    """
    批量处理CSV文件中的输入并将结果保存
    
    Args:
        csv_path: CSV文件路径
        output_path: 输出文件路径，默认为原文件名_processed.csv
        input_column: 输入列名
        output_column: 输出列名
        batch_size: 批处理大小
        max_new_tokens: 最大生成长度
        temperature: 生成温度
    """
    try:
        # 设置默认输出路径
        if output_path is None:
            base_name = os.path.splitext(csv_path)[0]
            output_path = f"{base_name}_processed.csv"
        
        # 读取CSV文件
        logger.info(f"正在读取CSV文件: {csv_path}")
        df = pd.read_csv(csv_path)
        
        # 检查输入列是否存在
        if input_column not in df.columns:
            raise ValueError(f"输入列 '{input_column}' 在CSV文件中不存在")
        
        # 提取输入列数据
        input_texts = df[input_column].fillna("").tolist()
        total_rows = len(input_texts)
        
        logger.info(f"开始处理 {total_rows} 行数据，批次大小: {batch_size}")
        
        # 初始化模型
        engine = QwenInferenceEngine()
        
        # 批量处理数据并显示进度
        all_results = []
        for i in tqdm(range(0, total_rows, batch_size), desc="批处理进度"):
            batch_texts = input_texts[i:i+batch_size]
            batch_results = engine.generate_batch(
                batch_texts, 
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                batch_size=batch_size
            )
            all_results.extend(batch_results)
            
            # 定期保存中间结果，防止意外中断
            if i > 0 and i % 3 == 0:
                temp_df = df.copy()
                temp_df[output_column] = all_results + [None] * (total_rows - len(all_results))
                temp_df.to_csv(f"{output_path}.temp", index=False)
                logger.info(f"已保存中间结果 ({len(all_results)}/{total_rows})")
        
        # 添加结果列
        df[output_column] = all_results
        
        # 保存结果
        df.to_csv(output_path, index=False)
        logger.info(f"处理完成，结果已保存至: {output_path}")
        
        # 删除临时文件
        if os.path.exists(f"{output_path}.temp"):
            os.remove(f"{output_path}.temp")
        
        return df
        
    except Exception as e:
        logger.error(f"处理CSV文件失败: {str(e)}")
        raise

# 使用示例
if __name__ == "__main__":
    csv_path = "/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/both_test_data.csv"
    processed_df = process_csv_file(
        csv_path=csv_path,
        batch_size=15,
        max_new_tokens=2048,
        temperature=0.4
    )
    print(f"成功处理 {len(processed_df)} 行数据")

/root/miniconda3/envs/news_emo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-03 16:49:38,171 - INFO - 正在读取CSV文件: /root/LLM_news_emo_analyze/DATA/both_model_train_data_0503/text_data_1000.xlsx
2025-05-03 16:49:38,174 - ERROR - 处理CSV文件失败: 'utf-8' codec can't decode byte 0x87 in position 10: invalid start byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 10: invalid start byte

In [9]:
import os
import torch
import logging
import pandas as pd
import numpy as np
import gc
from typing import List, Dict, Optional
from modelscope import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 模型默认路径
DEFAULT_MODEL_PATH = "/root/LLM_news_emo_analyze/Qwen_output/0430_merge_model_2100"

class QwenInferenceEngine:
    """Qwen模型推理引擎，支持批量处理和内存管理"""
    
    def __init__(self, model_path=None, config=None):
        # 统一配置参数
        self.config = {
            "max_new_tokens": 3000,       # 最大生成长度
            "temperature": 0.4,          # 温度参数
            "top_p": 0.9,                # 采样参数
            "repetition_penalty": 1.1,   # 重复惩罚系数
            "batch_size": 15,             # 批处理大小
            "max_input_length": 2048,    # 最大输入长度
            "clear_cuda_cache": True,    # 是否清理CUDA缓存
            "dynamic_batch_size": True,  # 是否启用动态批处理大小
            "min_batch_size": 12,         # 最小批处理大小
            "cuda_empty_freq": 1         # 每处理多少批次清空一次CUDA缓存
        }
        
        # 更新用户提供的配置
        if config:
            self.config.update(config)
            
        # 设置模型路径
        self.model_path = model_path or os.environ.get("QWEN_MODEL_PATH", DEFAULT_MODEL_PATH)
        
        self._load_model()
    
    def _load_model(self):
        """加载模型和分词器"""
        try:
            # 先清理环境
            self._clear_cuda_cache()
            
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_path,
                padding_side='left',
                trust_remote_code=True
            )
            
            # 检测可用GPU并设置合适的精度
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            if self.device == "cpu":
                logger.warning("未检测到GPU，将使用CPU进行推理，性能可能受限")
                torch_dtype = torch.float32
            else:
                logger.info(f"检测到{torch.cuda.device_count()}个GPU设备")
                torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
            
            # 动态分配GPU内存
            max_memory = {}
            if torch.cuda.is_available():
                for i in range(torch.cuda.device_count()):
                    gpu_total_mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)  # GB
                    max_memory[i] = f"{int(gpu_total_mem * 0.8)}GB"  # 使用80%的GPU内存，留余量防止OOM
            
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                torch_dtype=torch_dtype,
                device_map="auto",
                max_memory=max_memory if max_memory else None,
                trust_remote_code=True
            ).eval()
            
            # 验证模型是否正常工作
            self._validate_model()
            
            logger.info(f"成功加载Qwen模型，设备:{self.device}，精度:{torch_dtype}")
            
        except Exception as e:
            logger.error(f"模型初始化失败: {str(e)}")
            self._clear_cuda_cache()  # 失败时也清理缓存
            raise RuntimeError(f"模型初始化失败: {str(e)}")
    
    def _validate_model(self):
        """验证模型是否正常加载和工作"""
        try:
            # 简单测试
            test_input = "Hello"
            input_ids = self.tokenizer(test_input, return_tensors="pt").input_ids.to(self.model.device)
            self.model.generate(input_ids, max_new_tokens=5)
            logger.info("模型验证通过")
            
            # 清理测试产生的缓存
            self._clear_cuda_cache()
            
        except Exception as e:
            logger.error(f"模型验证失败: {str(e)}")
            self._clear_cuda_cache()
            raise RuntimeError(f"模型验证失败: {str(e)}")
    
    def _clear_cuda_cache(self):
        """清理CUDA缓存释放内存"""
        if not torch.cuda.is_available() or not self.config["clear_cuda_cache"]:
            return
            
        # 确保所有GPU操作完成
        torch.cuda.synchronize()
        
        # 手动触发垃圾回收
        gc.collect()
        
        # 清空CUDA缓存
        with torch.cuda.device("cuda"):
            torch.cuda.empty_cache()
            
        # 检查并报告当前内存使用情况
        if torch.cuda.is_available():
            for i in range(torch.cuda.device_count()):
                mem_allocated = torch.cuda.memory_allocated(i) / (1024**3)  # GB
                mem_reserved = torch.cuda.memory_reserved(i) / (1024**3)  # GB
                logger.debug(f"GPU:{i} 分配内存:{mem_allocated:.2f}GB, 保留内存:{mem_reserved:.2f}GB")
    
    def generate_response(self, prompt, **kwargs):
        """单条输入的推理函数，保持向后兼容性"""
        results = self.generate_batch([prompt], **kwargs)
        return results[0] if results else None
    
    def generate_batch(self, prompts, **kwargs):
        """批量处理多个输入的推理函数"""
        if not prompts:
            logger.warning("收到空输入列表")
            return []
        
        # 合并配置参数，允许临时覆盖默认配置
        config = self.config.copy()
        if kwargs:
            config.update(kwargs)
        
        results = []
        batch_size = config["batch_size"]
        batch_counter = 0
        
        # 分批处理
        for i in range(0, len(prompts), batch_size):
            batch_prompts = prompts[i:i+batch_size]
            
            # 处理批次
            try:
                batch_results = self._process_single_batch(batch_prompts, config)
                results.extend(batch_results)
                
                # 增加计数器
                batch_counter += 1
                
                # 定期清理内存
                if config["clear_cuda_cache"] and batch_counter % config["cuda_empty_freq"] == 0:
                    logger.debug(f"清理CUDA缓存（第{batch_counter}批次后）")
                    self._clear_cuda_cache()
                    
            except RuntimeError as e:
                # 处理CUDA内存不足错误
                if "CUDA out of memory" in str(e) and config["dynamic_batch_size"] and batch_size > config["min_batch_size"]:
                    # 清理内存
                    self._clear_cuda_cache()
                    
                    # 减小批处理大小
                    new_batch_size = max(batch_size // 2, config["min_batch_size"])
                    logger.warning(f"CUDA内存不足，减小批处理大小: {batch_size} -> {new_batch_size}")
                    batch_size = new_batch_size
                    config["batch_size"] = new_batch_size
                    
                    # 重试当前批次
                    i -= batch_size  # 回退以重新处理当前批次
                    continue
                else:
                    # 其他错误，清理内存并重新抛出
                    self._clear_cuda_cache()
                    logger.error(f"批处理错误: {str(e)}")
                    # 返回对应数量的None
                    batch_results = [None] * len(batch_prompts)
                    results.extend(batch_results)
            
        # 完成所有批次后清理内存
        self._clear_cuda_cache()
        return results
    
    def _process_single_batch(self, batch_prompts, config):
        """处理单批次输入"""
        try:
            # 构建消息模板
            batch_messages = [
                [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ] for prompt in batch_prompts
            ]
            
            # 应用聊天模板
            batch_texts = [
                self.tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                ) for messages in batch_messages
            ]
            
            # 统一处理tokenization
            encodings = self.tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=config["max_input_length"],
                return_attention_mask=True
            ).to(self.model.device)
            
            # 生成配置
            generation_config = {
                "max_new_tokens": config["max_new_tokens"],
                "temperature": config["temperature"],
                "top_p": config["top_p"],
                "repetition_penalty": config["repetition_penalty"],
                "pad_token_id": self.tokenizer.eos_token_id,
                "attention_mask": encodings.attention_mask,
                "do_sample": config["temperature"] > 0.0
            }
            
            # 执行批量生成
            with torch.no_grad():
                generated_ids = self.model.generate(
                    encodings.input_ids,
                    **generation_config
                )
            
            # 提取生成结果
            batch_responses = []
            for i, output_ids in enumerate(generated_ids):
                response_ids = output_ids[len(encodings.input_ids[i]):]
                response_text = self.tokenizer.decode(response_ids, skip_special_tokens=True).strip()
                batch_responses.append(response_text)
            
            # 手动删除中间变量释放内存
            del encodings
            del generated_ids
            
            return batch_responses
            
        except Exception as e:
            # 转发错误以便上层处理OOM
            raise
        
def process_csv_file(csv_path, output_path=None, input_column="qwen_input", output_column="qwen_output", **kwargs):
    """
    批量处理CSV文件中的输入，只处理output_column为空的行
    
    Args:
        csv_path: CSV文件路径
        output_path: 输出文件路径，默认为原文件名_processed.csv
        input_column: 输入列名
        output_column: 输出列名
        **kwargs: 可选的配置参数，会覆盖默认配置
    """
    try:
        # 设置默认输出路径
        if output_path is None:
            base_name = os.path.splitext(csv_path)[0]
            output_path = f"{base_name}_processed.csv"
        
        # 读取CSV文件
        logger.info(f"正在读取CSV文件: {csv_path}")
        # df = pd.read_csv(csv_path,encoding = 'utf_8_sig',lineterminator='\n')
        df = pd.read_excel(csv_path)
        # df['qwen_score'] = None
        # df['qwen_output'] = None
        
        # 检查输入列是否存在
        if input_column not in df.columns:
            raise ValueError(f"输入列 '{input_column}' 在CSV文件中不存在")
        
        # 确保输出列存在
        if output_column not in df.columns:
            df[output_column] = None
        
        # 判断哪些行需要处理（output_column为空的行）
        empty_mask = df[output_column].isna() | (df[output_column] == "") | (df[output_column].astype(str) == "nan") | (df['qwen_score'].astype(str) == "nan")
        rows_to_process = df[empty_mask].index.tolist()
        
        total_rows = len(df)
        empty_rows = len(rows_to_process)
        
        logger.info(f"CSV文件共有 {total_rows} 行，其中 {empty_rows} 行需要处理")
        
        # 如果没有需要处理的行，直接返回
        if empty_rows == 0:
            logger.info("没有空值行需要处理，直接返回原文件")
            return df
        
        # 只提取需要处理的行的输入
        inputs_to_process = df.loc[rows_to_process, input_column].fillna("").tolist()
        
        # 设置更加保守的默认配置以避免OOM
        default_config = {
            "batch_size": 12,                # 默认较小批次避免OOM
            "clear_cuda_cache": True,       # 启用内存清理
            "dynamic_batch_size": True,     # 启用动态批大小
            "cuda_empty_freq": 1            # 每批次都清理
        }
        
        # 合并用户配置
        config = default_config.copy()
        config.update(kwargs)
        
        # 初始化模型引擎
        engine = QwenInferenceEngine(config=config)
        batch_size = engine.config["batch_size"]
        
        logger.info(f"开始处理 {empty_rows} 行空值数据，初始批次大小: {batch_size}")
        
        # 批量处理数据并显示进度
        results_map = {}  # 使用字典存储结果，键为行索引
        processed_count = 0
        
        # 尝试从临时文件恢复部分结果
        if os.path.exists(f"{output_path}.temp"):
            try:
                temp_df = pd.read_csv(f"{output_path}.temp")
                if output_column in temp_df.columns:
                    # 只复制已处理的行
                    for idx in rows_to_process:
                        if idx < len(temp_df) and not pd.isna(temp_df.loc[idx, output_column]) and temp_df.loc[idx, output_column] != "":
                            results_map[idx] = temp_df.loc[idx, output_column]
                            processed_count += 1
                    
                    if processed_count > 0:
                        logger.info(f"从临时文件恢复了 {processed_count}/{empty_rows} 行结果")
            except Exception as temp_e:
                logger.warning(f"读取临时文件失败: {str(temp_e)}")
        
        # 更新需要处理的行
        rows_to_process = [idx for idx in rows_to_process if idx not in results_map]
        inputs_to_process = df.loc[rows_to_process, input_column].fillna("").tolist()
        
        logger.info(f"还需处理 {len(rows_to_process)} 行数据")
        
        try:
            # 按批次处理
            for i in tqdm(range(0, len(rows_to_process), batch_size), desc="批处理进度"):
                batch_indices = rows_to_process[i:i+batch_size]
                batch_inputs = inputs_to_process[i:i+batch_size]
                
                # 处理一批数据
                batch_results = engine.generate_batch(batch_inputs)
                
                # 将结果与原始行索引对应起来
                for j, result in enumerate(batch_results):
                    idx = batch_indices[j]
                    results_map[idx] = result
                
                processed_count += len(batch_indices)
                
                # 定期保存中间结果
                if i > 0 and i % 50 == 0:
                    # 将已处理的结果更新到DataFrame
                    temp_df = df.copy()
                    for idx, result in results_map.items():
                        temp_df.loc[idx, output_column] = result
                    
                    # 保存临时文件
                    temp_df.to_csv(f"{output_path}.temp", index=False,encoding = 'utf_8_sig')
                    logger.info(f"已保存中间结果 ({processed_count}/{empty_rows} 行)")
        
        except KeyboardInterrupt:
            logger.warning("用户中断处理，保存当前进度")
            
        finally:
            # 将所有结果更新到原始DataFrame
            for idx, result in results_map.items():
                df.loc[idx, output_column] = result
            
            # 保存最终结果
            df.to_csv(output_path, index=False)
            logger.info(f"处理完成，共处理 {len(results_map)}/{empty_rows} 行，结果已保存至: {output_path}")
            
            # 删除临时文件
            if os.path.exists(f"{output_path}.temp"):
                os.remove(f"{output_path}.temp")
            
            # 清理资源
            engine._clear_cuda_cache()
            del engine
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        return df
        
    except Exception as e:
        logger.error(f"处理CSV文件失败: {str(e)}")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        raise

# 向后兼容的全局函数
_global_engine = None

def generate_response(prompt, max_length=512, temperature=0.5):
    """兼容原始API的推理函数"""
    global _global_engine
    if _global_engine is None:
        _global_engine = QwenInferenceEngine()
    return _global_engine.generate_response(prompt, max_new_tokens=max_length, temperature=temperature)

# 使用示例
if __name__ == "__main__":
    csv_path = "/root/LLM_news_emo_analyze/DATA/both_model_train_data_0503/data1000.xlsx"
    
    # 设置更保守的参数以避免OOM
    config = {
        "max_new_tokens": 4096,
        "temperature": 0.3,
        "batch_size": 12,              # 初始批处理大小
        "clear_cuda_cache": True,     # 启用内存清理
        "dynamic_batch_size": True,   # 启用动态批处理大小调整
        "min_batch_size": 12,          # 最小可降至12条处理
        "cuda_empty_freq": 1          # 每批次后清理
    }
    
    processed_df = process_csv_file(
        csv_path=csv_path,
        input_column="qwen_input",
        output_column="qwen_output",
        **config
    )
    
    # 输出处理统计
    empty_after = processed_df["qwen_output"].isna().sum()
    filled = len(processed_df) - empty_after
    print(f"处理完成: 共 {len(processed_df)} 行，已填充 {filled} 行，剩余空值 {empty_after} 行")

/root/miniconda3/envs/news_emo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-03 19:13:51,594 - INFO - 正在读取CSV文件: /root/LLM_news_emo_analyze/DATA/both_model_train_data_0503/data1000.xlsx
2025-05-03 19:13:52,156 - INFO - CSV文件共有 1000 行，其中 1 行需要处理
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-05-03 19:13:53,058 - INFO - 检测到1个GPU设备
2025-05-03 19:13:56,193 - INFO - 模型验证通过
2025-05-03 19:13:56,316 - INFO - 成功加载Qwen模型，设备:cuda，精度:torch.bfloat16
2025-05-03 19:13:56,317 - INFO - 开始处理 1 行空值数据，初始批次大小: 12
2025-05-03 19:13:56,319 - INFO - 还需处理 1 行数据
批处理进度: 100%|██████████| 1/1 [00:34<00:00, 34.14s/it]
2025-05-03 19:14

处理完成: 共 1000 行，已填充 1000 行，剩余空值 0 行


### **qwen的score提取以及合理值检验**

In [10]:
import pandas as pd
test = pd.read_csv('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0503/data1000_processed.csv',encoding = 'utf_8_sig',lineterminator='\n')

In [11]:
import re
def extract_score_by_regex(text):
    if not isinstance(text, str):
        return None
    # 匹配 "情绪得分":"1.0" 或 "情绪得分": "0.5" 等格式
    match = re.search(r'"情绪得分"\s*:\s*"?(?P<score>-?1\.0|-?0\.5|0\.5|0\.0|0|1\.0|1)"?', text)
    if match:
        return match.group("score")
    return None

# 应用到 DataFrame
test['qwen_score'] = test['qwen_output'].apply(extract_score_by_regex)

In [12]:
valid_scores = {-1.0, -0.5, 0.0, 0.5, 1.0}

# 转换为 float，确保是数值型
test['qwen_score'] = pd.to_numeric(test['qwen_score'], errors='coerce')

# 检查是否存在非法得分
invalid_scores = test[~test['qwen_score'].isin(valid_scores)]

# 输出结果
if invalid_scores.empty:
    print("✅ 所有 qwen_score 的值都合法。")
else:
    print(f"❌ 存在非法得分，共 {len(invalid_scores)} 条记录：")
    print(invalid_scores[['qwen_output', 'qwen_score']].head())

✅ 所有 qwen_score 的值都合法。


In [7]:
test.iloc[225,-6]

'{"情绪分析":"好的，我现在需要处理这个用户的查询，分析给定的新闻文本的情绪，并给出五档制评分。首先，我要仔细阅读用户提供的所有信息，确保自己完全理解任务的要求。\n\n首先看新闻文本：“通过国家医保谈判，4款国产药物的降价幅度高达80%以上。” 这句话的关键点在于国家医保谈判带来的药品降价效果显著，具体来说是“高达80%以上”。这里有两个主要部分：谈判过程以及降价幅度。接下来要结合提供的情绪词典来进行分析。\n\n情绪词典中有两个词：markdown:0.0 和 undercut:-0.3。我需要检查新闻文本中是否有这些关键词或者相关词汇。原文提到的是“降价幅度高达80%以上”，这里的“降价”可能对应到情绪词典里的“undercut”吗？不过情绪词典里并没有这个词，所以可能需要考虑其他因素。\n\n根据评分逻辑，首先要进行语义匹配，忽略不相关的词汇，只保留有效情绪词。然后整体语义优先，情绪词辅助修正。虽然情绪词典中没有直接匹配的词，但需要看整体内容是否带有明显的积极或消极情绪。\n\n新闻中的核心信息是国产药物通过医保谈判获得了大幅度降价，这通常被视为积极的消息，因为它降低了患者用药的成本，可能促进医疗可及性和公平性。然而，情绪词典中的“undercut”是指压价销售，而文中提到的是降价幅度高，可能并不等同于压价行为，因为降价本身是积极措施。但情绪词典里只有undercut:-0.3，所以如果这里确实出现了“降价”这样的词汇，那么可能需要考虑是否属于undercut的范畴。\n\n但用户提供的情绪词典中undercut的分值是-0.3，而markdown是0.0。在新闻文本中，有没有出现这两个词呢？原文中的“降价”是否对应undercut？可能不太直接，因为undercut更多指主动压价，而这里只是被动降价，所以可能不算undercut。这时候需要判断整体语义是否积极。\n\n根据示例中的情况，比如示例5中虽然没有直接匹配情绪词，但整体语义非常负面，所以评分为-1。同样，如果整体语义足够积极，即使没有直接的情绪词匹配，也可能评为较高分。例如示例1中，净利润大幅增长，直接匹配了profit和profitability，所以得了1分。而示例2中，虽然有间接映射，但因为没有直接匹配，所以得0.5分。\n\n在这个案例中，新闻内容明显是关于降价的积极消息，尤其是“高达

In [14]:
test.to_excel('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0503/data1000_process.xlsx')

In [23]:
test.to_csv('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/both_test_data_processed_222.csv',index = False, encoding = 'utf_8_sig')

### **结果准确率评估**

In [1]:
import pandas as pd
import re
test = pd.read_csv('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/wxy_change_label.csv',encoding = 'utf_8_sig',lineterminator='\n')

In [ ]:
### ds_api的score提取

def extract_sentiment_score(df, text_column, new_column):
    # 正则表达式兼容两种引号情况
    pattern = r'(?:"情绪得分":(?: "|")([-\d.]+)(?:"))'
    df[new_column] = df[text_column].apply(
        lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else None)
    return df

# 调用函数提取情绪得分
test = extract_sentiment_score(test, 'content', 'ds_api_score')

In [16]:
test

,Unnamed: 0,Title,NewsSource,yearmonth,stratum,topic,similarity,prompt,cut_news,sentiment_dict,...,qwen_input,Response,Complex_CoT,label,Question,qwen_output,qwen_score,content,reasoning_content\r,ds_api_score
0,11,评论：一份有情怀有思路的政府报告,第一财经日报,15-Mar,2015-03-06 00:00:00_第一财经日报,“宏观经济”,0.543562,新闻文本是##“总理的报告，在解题思路方面亮点纷呈”##，包含的情绪词典是##：解决/解答(...,总理的报告，在解题思路方面亮点纷呈,"解决/解答(solve):0.3,问题/难题(problem):-0.4",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""0.5""}",好的，我现在需要分析这段新闻文本的情绪得分。首先，新闻内容是关于总理的报告在解题思路方面有亮...,0.5,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,"{""情绪分析"":""好的，我现在需要处理这个用户的查询，分析给定的新闻文本的情绪得分。首先，用...",1.0,"{""情绪分析"":""①语义信息强调总理报告的解题思路存在'亮点纷呈'，突出其策略创新性与解决方...",好的，我现在需要分析这个新闻文本的情绪，并根据给定的情绪词典给出评分。首先，新闻文本是：“总...,0.5
1,6,基金年度“成绩单”角逐进入白热化 绩优产品多布局传媒消费通讯等领域,经济参考报,23-Nov,2023-11-28 00:00:00_经济参考报,“金融”,0.501491,新闻文本是##“截至目前，共989只基金成立以来的回报率在100%以上”##，包含的情绪词典...,截至目前，共989只基金成立以来的回报率在100%以上,"百分比(percentage):0.0,基金/资金(fund):0.1",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""0.5""}",好的，我来仔细分析一下这个新闻文本的情绪。首先，新闻内容是：“截至目前，共989只基金成立以...,0.5,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,"{""情绪分析"":""好的，我需要分析这条新闻的情绪，并给出五档评分。首先看新闻内容：“截至目前...",0.5,"{""情绪分析"":""①语义信息为近千只基金成立以来的回报率超过100%，展示出较强的长期收益能...",好的，我需要分析这条新闻文本的情绪，并给出对应的五档制评分。首先，看看新闻内容：“截至目前，...,0.5
2,0,元旦谁抢中国游客多？日本韩国，一个笑了一个哭了,每日经济新闻,18-Jan,2018-01-02 00:00:00_每日经济新闻,“消费”,0.611313,新闻文本是##“携程跟团游、自由行数据，香港、曼谷、东京、新加坡、大阪、长滩岛、台北、芽庄、...,携程跟团游、自由行数据，香港、曼谷、东京、新加坡、大阪、长滩岛、台北、芽庄、迪拜、伊斯坦布尔...,"海外的/国外的(overseas):0.0,外国的/国外的(foreign):0.0",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""0""}",好的，我现在需要分析这条新闻的情绪得分。首先，看看新闻内容。主要讲的是携程的跟团游和自由行数...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,"{""情绪分析"":""好的，我现在需要处理这个用户的查询，分析给定的新闻文本的情绪，并给出五档制...",1.0,"{""情绪分析"":""①语义信息为列举中国游客境外热门旅游目的地及购物季对城市人气的提升效应，属...",好的，我现在需要分析这个新闻文本的情绪，并根据给定的任务流程和评分逻辑给出结果。首先，我要仔...,0.0
3,8,又一大省放开落户！除两城外，山东其余14城或全部“零门槛”,21世纪经济报道,20-Jan,2020-01-13 00:00:00_21世纪经济报道,“房地产”,0.534588,新闻文本是##“生产要素包括人口的流动还会更加自由，所以城镇化绝对不是一个就地城镇化的概念”...,生产要素包括人口的流动还会更加自由，所以城镇化绝对不是一个就地城镇化的概念,"自由的/免费的(free):0.2,非流动性/缺乏流动性(illiquidity):-0.5",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""0""}",好的，我现在需要分析给定的新闻文本的情绪得分。首先，看一下新闻内容：“生产要素包括人口的流动...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,"{""情绪分析"":""好的，我现在需要分析给定的新闻文本的情绪，并根据提供的情绪词典给出五档制评...",0.5,"{""情绪分析"":""①语义信息强调生产要素与人口流动将更加自由，并明确否定就地城镇化的局限性，...",好的，我现在需要分析这条新闻文本的情绪，并给出对应的五档制评分。首先，我得仔细阅读新闻内容，...,0.5
4,12,专访顾云昌：北京共有产权房是1998年房改后的又一重大创新,中国经济周刊,17-Aug,2017-08-11 17:35:00_中国经济周刊,“房地产”,0.549985,新闻文本是##“而保障房可分为两种：公租房，以及新推出的共有产权房”##，包含的情绪词典是#...,而保障房可分为两种：公租房，以及新推出的共有产权房,"房子/住宅(house):0.0,房地产/财产(estate):0.0",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""0""}",好，我需要分析这段关于中国保障房的新闻文本的情绪。首先，看整体内容。新闻提到保障房分为公租房...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,"{""情绪分析"":""好的，我现在需要处理用户提供的这个市场情绪分析任务。首先，我得仔细阅读用户...",0.0,"{""情绪分析"":""①语义信息为对保障房类型的客观分类说明，未体现政策效果或市场影响的价值判断...",好的，我现在要处理这条新闻文本的情绪分析任务。首先，我需要仔细阅读新闻内容：“而保障房可分为...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,492,赢创：创新环保及城镇化将成为今后发展三大主题,第一财经日报,13-Apr,2013-04-03 00:00:00_第一财经日报,“生产投资”,0.482415,新闻文本是##“环保标准全球统一\n 研发新产品是一家创新型特种化学品企业的核心要素之一”...,环保标准全球统一\n 研发新产品是一家创新型特种化学品企业的核心要素之一,"创新/革新(innovation):0.7,创新性/革新性(innovativeness):0.8",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""0.5""}",好的，我现在需要处理这个用户的查询。首先，用户给了一个任务，要求作为中英双语市场情绪分析专家...,0.0,**角色定义**\n擅长中英双语的市场情绪分析专家，能够对中国市场的新闻文本情绪进行准确的评...,"{""情绪分析"":""好的，我现在需要处理这个用户的查询，分析给定的新闻文本的情绪，并给出五档制...",0.5,"{""情绪分析"":""①语义信息为强调环保标准全球统一及创新型企业的研发重要性，前者隐含标准化带...",好，我现在要分析这个新闻文本的情绪得分。首先，新闻文本是：“环保标准全球统一 研发新产品是一...,0.5
494,482,“七连涨”遇大跳水 融资盘企稳再受考验,第一财经日报,15-Aug,2015-08-19 00:00:00_第一财经日报,“金融”,0.546733,新闻文本是##“当8月18日大盘调整再度来袭时，资金面信心也将再次接受严峻考验”##，包含的...,当8月18日大盘调整再度来袭时，资金面信心也将再次接受严峻考验,"资金不足的(underfunded):-0.3,衰退/不景气(recession):-0.95",...,**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的...,"{""情绪得分"": ""-0.5""}",好的，我需要分析这条新闻的情绪，并给出五档评分。首先看新闻内容：“当8月18日大盘调整再度

In [20]:
pattern = r'"情绪得分": "(-?\d+(?:\.\d+)?)"'
def extract_score(text):
    match = re.search(pattern, text)
    if match:
        return (match.group(1))
    return None

# 应用函数到 Response 列
test['ds_score'] = test['Response'].apply(extract_score)

In [21]:
valid_scores = {-1.0, -0.5, 0.0, 0.5, 1.0}

# 转换为 float，确保是数值型
test['ds_score'] = pd.to_numeric(test['ds_score'], errors='coerce')

# 检查是否存在非法得分
invalid_scores = test[~test['ds_score'].isin(valid_scores)]

# 输出结果
if invalid_scores.empty:
    print("✅ 所有 ds_api_sds_scorecore 的值都合法。")
else:
    print(f"❌ 存在非法得分，共 {len(invalid_scores)} 条记录：")
    print(invalid_scores[['qwen_output', 'ds_score']].head())

✅ 所有 ds_api_sds_scorecore 的值都合法。


In [13]:
import pandas as pd

def count_matching_rows(test, ds_score_col, qwen_score_col, special_treatment_for_zero=True):
    if special_treatment_for_zero:
        # 定义条件，考虑对 0 的特殊处理
        conditions = (
            ((test[ds_score_col] == 0) & (test[qwen_score_col] == 0)) | 
            ((test[ds_score_col] != 0) & ((test[ds_score_col] - test[qwen_score_col]).abs() <= 0.5))
        )
    else:
        # 不考虑对 0 的特殊处理，仅判断差值绝对值
        conditions = (test[ds_score_col] - test[qwen_score_col]).abs() <= 0.5

    # 筛选符合条件的行
    matching_rows = test[conditions]

    # 计算匹配行数
    matching_count = len(matching_rows)
    print(matching_count/498)

    return matching_count

count_matching_rows(test,'label','qwen_score', special_treatment_for_zero=False)

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [5]:
test.to_excel('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/wxy_change_label.xlsx')

In [13]:
prompt_template = """**角色定义**
你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的新闻文本情绪进行准确的评分：

**任务流程**
1. 分析新闻整体内容情绪
2. 匹配情绪词典关键词，对情绪进行深入理解与分析
3. 输出情绪分析跟对应的五档制评分结果（-1.0/-0.5/0.0/0.5/1.0，分别代表非常消极、比较消极、中性、比较积极、非常积极）

**评分逻辑**
1. 语义匹配：忽略偏离的情绪词词汇，保留有效情绪词
2. 评分调整：整体语义优先，情绪词辅助修正

**示例说明**
- 示例1：
新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一
情绪词典：profitability:0.6,profit:0.8
情绪分析:①语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破；②关键情绪词调整：“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1.0分（非常积极）的评分结果。
情绪得分:1.0

- 示例2：
新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力
情绪词典：mispricing:-0.4,advantage:0.7
情绪分析:①语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号；②关键情绪词调整：未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。由于缺乏词典强匹配项限制进一步上调空间，整体乐观基调符合“比较积极”，情绪得分0.5。
情绪得分:0.5

- 示例3：
新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动
情绪词典：free:0.2,immateriality:-0.2
情绪分析:①语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响；②关键情绪词调整：“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriality”原词（仅隐含“无形的手”概念），语义匹配强度不足。陈述性内容缺乏明确情绪导向，符合中性基准0.0分。
情绪得分:0.0

- 示例4：
新闻文本：我们投入的前期费用谁来承担
情绪词典：invest:0.3
情绪分析:①语义信息为对前期费用承担主体的质疑，隐含投入成本未被消化的潜在风险，传递财务负担不确定性的负面情绪；②关键情绪词调整：“投入”（匹配invest:+0.3）存在方向性冲突，因文本中“投入”实际指向成本分摊压力而非正向投资预期，情绪词得分被整体语义逆向修正。中性词主导+隐含担忧的复合信号符合低度负面评分档位“比较消极”，即情绪得分-0.5分。
情绪得分:-0.5

- 示例5：
新闻文本：饱受美国次贷危机冲击的华尔街再次风云突变
情绪词典：crash:-0.9,meltdown:-0.8
情绪分析:①语义信息为华尔街受次贷危机冲击引发的市场动荡，此类全球金融中心的不稳定通常导致跨国资本避险情绪上升，对中国市场构成外溢风险；②关键情绪词调整：未直接匹配“crash”或“meltdown”，但“次贷危机冲击”与“风云突变”共同映射系统性风险（贴近meltdown的-0.8方向），叠加事件严重性突破常规调整范畴。极端负面事件的整体语义强度主导评分，因此是非常消极，情绪得分-1.0分。
情绪得分:-1.0


**其他说明**
- 情绪词典的分值仅作语义方向参考
- 情绪得分必须是五档制选择，不得出现-1.0/-0.5/0.0/0.5/1.0之外的分数
- 输出格式：{"情绪分析":"...","情绪得分":"..."}

现在，请你开始分析并按照要求输出结果：
新闻文本：{cut_news}
情绪词典：{sentiment_dict_v2}"""

In [14]:
sampled_data['qwen_input'] = None

# 循环遍历每一行
for index, row in sampled_data.iterrows():
    cut_news = row['cut_news']
    sentiment_dict_v2 = row['sentiment_dict_v2']
    filled_template = prompt_template.replace('{cut_news}', cut_news).replace('{sentiment_dict_v2}', sentiment_dict_v2)
    sampled_data.at[index, 'qwen_input'] = filled_template

In [7]:
import pandas as pd
import re
test1 = pd.read_csv('/root/LLM_news_emo_analyze/DATA/qwen_sft_train_0330data/qwen_test_data.csv',encoding = 'utf_8_sig',lineterminator='\n')
test2 = pd.read_csv('/root/LLM_news_emo_analyze/DATA/both_model_train_data_0430/both_test_data_processed_2_processed_processed.csv',encoding = 'utf_8_sig',lineterminator='\n')

In [ ]:
# 找出在 test1 但不在 test2 中的数据
unique_data = test1[~test1['cut_news'].isin(test2['cut_news'])]

# 随机抽取 502 条数据
if len(unique_data) >= 502:
    sampled_data = unique_data.sample(n=502)
else:
    print("满足条件的数据不足 502 条，只能抽取 {} 条数据。".format(len(unique_data)))
    sampled_data = unique_data

              DeclareDate                                      Title  \
2683  2018-04-02 00:00:00                                第三批自贸试验区周年考   
2689  2016-10-31 15:42:00                         5575万人脱贫的“硬骨头”如何啃?   
2577  2019-07-05 23:34:00                              “跟投大法”大规模受阻背后   
1048  2019-03-02 00:00:00           外卖用户规模已高达3.6亿新零售跨界致外卖市场竞争格局进入新阶段   
2568  2019-03-12 00:00:00                                 补齐农村基础设施短板   
...                   ...                                        ...   
2019  2010-01-15 00:00:00                         褐皮书“示好” 美联储或布局退出政策   
941   2011-04-07 00:00:00                            鼎立股份稀土概念“画饼充饥”？   
2533  2023-08-02 00:00:00                           天健集团：接受财通证券等机构调研   
1984  2015-04-03 00:00:00                                      ■行业资讯   
1459  2017-01-09 00:00:00  金融危机以来全球690次降息后 2017转变开始：货币刺激退居二线财政政策走上前台   

                                            NewsContent NewsSource yearmonth  \
2683  最是一年春好处，何况是周年到来之日。2017年4月1日，中国第三批自贸试验区挂牌，吹响了新一...

In [10]:
test2.columns

Index(['Unnamed: 0', 'Title', 'NewsSource', 'yearmonth', 'stratum', 'topic',
       'similarity', 'prompt', 'cut_news', 'sentiment_dict',
       'sentiment_dict_v2', 'qwen_input', 'Response', 'Complex_CoT', 'label',
       'Question', 'qwen_output', 'qwen_score'],
      dtype='object')

In [ ]:
same_col = ['Title', 'NewsSource', 'yearmonth', 'stratum', 'topic',
       'similarity', 'prompt', 'cut_news', 'sentiment_dict',
       'sentiment_dict_v2','']

In [ ]:
test1 = test1[]

Index(['DeclareDate', 'Title', 'NewsContent', 'NewsSource', 'yearmonth',
       'stratum', 'topic', 'similarity', 'prompt', 'cut_news',
       'sentiment_dict', 'sentiment_dict_v2', 'qwen_input'],
      dtype='object')

In [17]:
sampled_data.iloc[2,-1]

'**角色定义**\n你是一位擅长中英双语的中国市场情绪分析专家，能够基于任务流程对中国市场的新闻文本情绪进行准确的评分：\n\n**任务流程**\n1. 分析新闻整体内容情绪\n2. 匹配情绪词典关键词，对情绪进行深入理解与分析\n3. 输出情绪分析跟对应的五档制评分结果（-1.0/-0.5/0.0/0.5/1.0，分别代表非常消极、比较消极、中性、比较积极、非常积极）\n\n**评分逻辑**\n1. 语义匹配：忽略偏离的情绪词词汇，保留有效情绪词\n2. 评分调整：整体语义优先，情绪词辅助修正\n\n**示例说明**\n- 示例1：\n新闻文本：实现净利润同比增长137.98%，单季度的盈利规模超过中信证券成为业内第一\n情绪词典：profitability:0.6,profit:0.8\n情绪分析:①语义信息为净利润同比大幅增长137.98%及单季度盈利规模跃居行业第一，均体现超预期的盈利能力突破；②关键情绪词调整：“净利润”（匹配profit）和“盈利”（匹配profitability）共同强化积极方向。两重强信号叠加符合最高档1.0分（非常积极）的评分结果。\n情绪得分:1.0\n\n- 示例2：\n新闻文本：中国的A股定位反而是比较便宜的，外资从全球定价认为我们非常有吸引力\n情绪词典：mispricing:-0.4,advantage:0.7\n情绪分析:①语义信息为A股估值被强调为“便宜”及外资认可其全球定价吸引力，隐含市场价值被低估的积极信号；②关键情绪词调整：未直接匹配词典中的“mispricing”或“advantage”，但“便宜”隐含定价偏离逻辑（映射mispricing方向），“有吸引力”间接呼应优势（advantage方向）。由于缺乏词典强匹配项限制进一步上调空间，整体乐观基调符合“比较积极”，情绪得分0.5。\n情绪得分:0.5\n\n- 示例3：\n新闻文本：美国信奉自由市场经济理念，主张靠无形的手调整经济活动\n情绪词典：free:0.2,immateriality:-0.2\n情绪分析:①语义信息为对美国经济理念的中性陈述，既未直接关联中国市场优劣，也未体现政策对华影响；②关键情绪词调整：“自由”（匹配free:+0.2）与“无形”（匹配immateriality:-0.2）存在方向冲突，但文本未实际使用“immateriali